### 🧹 Limpieza de etiquetas YOLO

Este script recorre las carpetas `train`, `valid` y `test` de un dataset con estructura YOLO. Para cada imagen:

- Verifica si existe su archivo de etiquetas.
- Elimina las clases 0 y 2.
- Reasigna la clase 1 a 0 y la clase 3 a 1.
- Si una imagen no tiene etiquetas válidas después del filtro, elimina tanto la imagen como su archivo de etiquetas.

Con esto se reorganizan las clases y se limpia el dataset automáticamente.


In [ ]:
import os
import shutil

def process_labels(images_dir, labels_dir):
    # Listar todas las imágenes
    for img_file in os.listdir(images_dir):
        if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            base_name = os.path.splitext(img_file)[0]
            txt_file = os.path.join(labels_dir, base_name + '.txt')
            
            # Si no existe el archivo de etiquetas, saltar
            if not os.path.exists(txt_file):
                continue
            
            # Leer y procesar etiquetas
            new_lines = []
            with open(txt_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    
                    if len(parts) != 5:
                        continue
                    
                    class_id = int(parts[0])
                    
                    # Eliminar clases 0 y 2
                    if class_id in {0, 2}:
                        continue
                    
                    # Cambiar clase 3 a 2
                    if class_id == 1:
                        parts[0] = '0'
                    if class_id == 3:
                        parts[0] = '1'
                    
                    new_lines.append(' '.join(parts))
            
            # Si no quedan etiquetas, eliminar archivos
            if not new_lines:
                os.remove(txt_file)
                img_path = os.path.join(images_dir, img_file)
                os.remove(img_path)
                print(f"Eliminados: {img_file} y su etiqueta")
            else:
                # Escribir nuevas etiquetas
                with open(txt_file, 'w') as f:
                    f.write('\n'.join(new_lines))

# Directorios base (cambiar según tu estructura)
data_dir = 'E:/Futsal/dataset'
splits = ['train', 'valid', 'test']

for split in splits:
    print(f"\nProcesando: {split}")
    images_dir = os.path.join(data_dir, split, 'images')
    labels_dir = os.path.join(data_dir, split, 'labels')
    
    if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
        print(f"Error: Directorios no encontrados para {split}")
        continue
    
    process_labels(images_dir, labels_dir)

print("\n¡Proceso completado!")

### 📊 Conteo de etiquetas YOLO

Este script recorre las carpetas `train`, `valid` y `test` de un dataset en formato YOLO y:

- Cuenta cuántas etiquetas hay por clase en cada subconjunto.
- Muestra cuántas imágenes hay por split.
- Ignora líneas mal formateadas.
- Al final, imprime un resumen total de etiquetas por clase y la cantidad total de etiquetas en todo el dataset.


In [3]:
import os
from collections import defaultdict

def count_labels(data_dir):
    splits = ['train', 'valid', 'test']
    total_counts = defaultdict(int)
    
    for split in splits:
        split_counts = defaultdict(int)
        labels_dir = os.path.join(data_dir, split, 'labels')
        
        if not os.path.exists(labels_dir):
            print(f"⚠️ Directorio no encontrado: {labels_dir}")
            continue
            
        num_images = len([f for f in os.listdir(os.path.join(data_dir, split, 'images')) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
        
        for label_file in label_files:
            with open(os.path.join(labels_dir, label_file), 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue  # Saltar líneas mal formateadas
                    class_id = parts[0]
                    split_counts[class_id] += 1
                    total_counts[class_id] += 1
        
        print(f"\n📊 Split: {split.upper()}")
        print(f"📁 Imágenes: {num_images}")
        print(f"🏷️ Etiquetas por clase:")
        for cls in sorted(split_counts, key=int):
            print(f"  Clase {cls}: {split_counts[cls]}")

    # Resultados totales
    print("\n🔍 TOTALES ACUMULADOS:")
    for cls in sorted(total_counts, key=int):
        print(f"  Clase {cls}: {total_counts[cls]}")
    print(f"🏷️ Total de etiquetas: {sum(total_counts.values())}")

# Configuración (cambiar por tu ruta)
data_dir = r'E:\Futsal\dataset'
# Ejecutar el conteo
count_labels(data_dir)


📊 Split: TRAIN
📁 Imágenes: 18277
🏷️ Etiquetas por clase:
  Clase 0: 20749
  Clase 1: 1859

📊 Split: VALID
📁 Imágenes: 3778
🏷️ Etiquetas por clase:
  Clase 0: 4147
  Clase 1: 477

📊 Split: TEST
📁 Imágenes: 3086
🏷️ Etiquetas por clase:
  Clase 0: 2554
  Clase 1: 760

🔍 TOTALES ACUMULADOS:
  Clase 0: 27450
  Clase 1: 3096
🏷️ Total de etiquetas: 30546


### 🖼️ Conversión y redimensionamiento de imágenes

Este script procesa todas las imágenes en una carpeta y realiza lo siguiente:

- Convierte cada imagen a escala de grises (blanco y negro).
- Redimensiona cada imagen a 640x640 píxeles.
- Sobrescribe la imagen original con la versión modificada.

In [ ]:
import os
from PIL import Image

# Ruta de la carpeta de imágenes

# Obtener lista de archivos de imágenes
images_dir = r"E:\Futsal\raw\images\test\images"


image_files = [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".png"))]  # Ajusta extensiones si es necesario

for image_file in image_files:
    image_path = os.path.join(images_dir, image_file)

    # Abrir imagen, convertir a escala de grises y redimensionar
    img = Image.open(image_path).convert("L").resize((640, 640))
    
    # Guardar la imagen modificada
    img.save(image_path)

print("Todas las imágenes han sido convertidas a blanco y negro y redimensionadas a 640x640 píxeles.")


Todas las imágenes originales han sido reemplazadas con su versión en blanco y negro.


### 🔀 Separación de datos de validación

Este script toma el 10% de las imágenes del conjunto de entrenamiento (`train`) y las mueve al conjunto de validación (`valid`), junto con sus archivos de etiqueta.

- Selecciona aleatoriamente el 10% de las imágenes en `train/images`.
- Mueve esas imágenes a `valid/images`.
- También mueve sus etiquetas correspondientes de `train/labels` a `valid/labels`.



In [1]:
import os
import shutil
import random

# Definir rutas
train_dir = r'E:\Futsal\FineTuning\dataset\train'
train_images_dir = os.path.join(train_dir, 'images')
train_labels_dir = os.path.join(train_dir, 'labels')

valid_dir = r'E:\Futsal\FineTuning\dataset\valid'
valid_images_dir = os.path.join(valid_dir, 'images')
valid_labels_dir = os.path.join(valid_dir, 'labels')

# Crear las carpetas de validación si no existen
os.makedirs(valid_images_dir, exist_ok=True)
os.makedirs(valid_labels_dir, exist_ok=True)

# Listar todas las imágenes en la carpeta de train/images
imagenes = os.listdir(train_images_dir)
total_imagenes = len(imagenes)

# Calcular el 10% de las imágenes
cantidad_valid = int(total_imagenes * 0.10)

# Seleccionar al azar las imágenes a mover
seleccionadas = random.sample(imagenes, cantidad_valid)

# Mover cada imagen y su label correspondiente
for imagen in seleccionadas:
    # Obtener el nombre base de la imagen (sin extensión)
    nombre_base, extension = os.path.splitext(imagen)
    
    # Rutas de origen para imagen y label
    origen_imagen = os.path.join(train_images_dir, imagen)
    origen_label = os.path.join(train_labels_dir, nombre_base + '.txt')
    
    # Rutas destino para imagen y label
    destino_imagen = os.path.join(valid_images_dir, imagen)
    destino_label = os.path.join(valid_labels_dir, nombre_base + '.txt')
    
    # Mover la imagen
    shutil.move(origen_imagen, destino_imagen)
    
    # Mover la etiqueta si existe
    if os.path.exists(origen_label):
        shutil.move(origen_label, destino_label)
    else:
        print(f"No se encontró el archivo de etiqueta para {imagen}")

print(f"Se movieron {cantidad_valid} imágenes y sus etiquetas a la carpeta '{valid_dir}'.")


Se movieron 102 imágenes y sus etiquetas a la carpeta 'E:\Futsal\FineTuning\dataset\valid'.
